In [1]:
import joblib
import pandas as pd
import numpy as np
from sklearn.utils import resample
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import root_mean_squared_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import VotingRegressor

In [2]:
# Set the directory where your files are located
directory = "C:/Users/diego/OneDrive - University of Florida/Desktop/UF/Washington/CommandersQuantTestData.csv"
# Load data
data = pd.read_csv(directory)
# Replace 'MISSING' with np.nan in the 'GAIN' column
data['GAIN'] = data['GAIN'].replace('MISSING', np.nan)
data.head()

,PLAYID,DOWN,DIST,LOS,GAIN,FORMATION,PLAYCALL,PLAYTYPE,PASSER,SCOREDIFF,DEFTEAM,KEY.PLAYER.NUMBER,KEY.PLAYER.POSITION,FUMBLE,INTERCEPTION,PASSRESULT
0,1,1.0,10.0,78,2,1x2 P,RUN 03 LT,RUN,Mikey Thomas,21,ORCAS,5.0,RB,True,False,NaN
1,2,1.0,10.0,80,-4,1x3,RUN 03 RT,RUN,Mikey Thomas,-14,HORNETS,5.0,RB,False,False,NaN
2,3,2.0,6.0,71,8,3x1,PROTECTION 03 PASS 24,PASS,Mikey Thomas,-18,FISHES,12.0,Q,False,False,SCRAMBLE
3,4,4.0,10.0,75,16,2x3,PROTECTION 03 PASS 20,PASS,Mikey Thomas,1,PONIES,19.0,WR1,False,False,COMPLETE
4,5,1.0,10.0,70,0,2x2,RUN 05 LT,RUN,Mikey Thomas,-14,HORNETS,5.0,RB,False,False,NaN


In [3]:
# Display summary statistics and data types
data.info()
data.describe()

# Check the unique values and data types in the 'GAIN' column
print("Unique values in 'GAIN' column:", data['GAIN'].unique())
print("Data types of columns:")
print(data.dtypes)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 625 entries, 0 to 624
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   PLAYID               625 non-null    int64  
 1   DOWN                 607 non-null    float64
 2   DIST                 607 non-null    float64
 3   LOS                  625 non-null    int64  
 4   GAIN                 500 non-null    object 
 5   FORMATION            625 non-null    object 
 6   PLAYCALL             625 non-null    object 
 7   PLAYTYPE             625 non-null    object 
 8   PASSER               625 non-null    object 
 9   SCOREDIFF            625 non-null    int64  
 10  DEFTEAM              625 non-null    object 
 11  KEY.PLAYER.NUMBER    582 non-null    float64
 12  KEY.PLAYER.POSITION  582 non-null    object 
 13  FUMBLE               625 non-null    bool   
 14  INTERCEPTION         625 non-null    bool   
 15  PASSRESULT           393 non-null    obj

In [4]:
# Identify rows with missing GAIN
missing_gain_rows = data[data['GAIN'].isna()]
missing_gain_indices = missing_gain_rows.index

# Display the shape of the dataset
print(f"Total rows: {data.shape[0]}, Rows with missing GAIN: {missing_gain_rows.shape[0]}")

Total rows: 625, Rows with missing GAIN: 125


In [5]:
# Impute missing values for DOWN and DIST using median in the original dataset
data['DOWN'] = data['DOWN'].fillna(data['DOWN'].median())
data['DIST'] = data['DIST'].fillna(data['DIST'].median())

# Display summary of imputed data
print("Summary statistics after imputing missing values for the entire dataset:")
data.describe()


Summary statistics after imputing missing values for the entire dataset:


,PLAYID,DOWN,DIST,LOS,SCOREDIFF,KEY.PLAYER.NUMBER
count,625.000000,625.000000,625.000000,625.000000,625.000000,582.000000
mean,313.000000,1.889600,9.035200,50.833600,-10.835200,18.647766
std,180.566239,0.943417,4.301018,24.559474,23.783614,23.649942
min,1.000000,1.000000,1.000000,1.000000,-72.000000,2.000000
25%,157.000000,1.000000,7.000000,32.000000,-26.000000,5.000000
50%,313.000000,2.000000,10.000000,56.000000,-14.000000,9.000000
75%,469.000000,2.000000,10.000000,72.000000,0.000000,19.000000
max,625.000000,4.000000,32.000000,98.000000,65.000000,93.000000


In [6]:
# Identify rows with missing GAIN
missing_gain_rows = data[data['GAIN'].isna()]
missing_gain_indices = missing_gain_rows.index

# Display the shape of the dataset
print(f"Total rows: {data.shape[0]}, Rows with missing GAIN: {missing_gain_rows.shape[0]}")

# Define numerical and categorical features
numeric_features = ['DOWN', 'DIST', 'LOS', 'SCOREDIFF']
categorical_features = ['FORMATION', 'PLAYCALL', 'PLAYTYPE', 'PASSER', 'DEFTEAM', 'KEY.PLAYER.POSITION', 'PASSRESULT']

# Handle outliers using the IQR method
def handle_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df[column] = np.where(df[column] < lower_bound, lower_bound, df[column])
    df[column] = np.where(df[column] > upper_bound, upper_bound, df[column])
    return df

# Apply outlier handling to numerical features
for feature in numeric_features:
    data = handle_outliers(data, feature)

# Display summary statistics after handling outliers
data.describe()


Total rows: 625, Rows with missing GAIN: 125


,PLAYID,DOWN,DIST,LOS,SCOREDIFF,KEY.PLAYER.NUMBER
count,625.000000,625.000000,625.000000,625.000000,625.000000,582.000000
mean,313.000000,1.852800,8.784000,50.833600,-11.436800,18.647766
std,180.566239,0.866911,3.185715,24.559474,22.156124,23.649942
min,1.000000,1.000000,2.500000,1.000000,-65.000000,2.000000
25%,157.000000,1.000000,7.000000,32.000000,-26.000000,5.000000
50%,313.000000,2.000000,10.000000,56.000000,-14.000000,9.000000
75%,469.000000,2.000000,10.000000,72.000000,0.000000,19.000000
max,625.000000,3.500000,14.500000,98.000000,39.000000,93.000000


In [7]:
import warnings
# Suppress specific warnings
warnings.filterwarnings("ignore", category=UserWarning, message="The least populated class in y has only 1 members")

# Impute and scale numerical features
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Impute and encode categorical features
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Separate data into known and unknown GAIN for training purposes
known_gain_data = data[data['GAIN'].notna()].copy()
known_gain_data['GAIN'] = known_gain_data['GAIN'].astype(float)

# Ensure there are actual missing values in the 'GAIN' column
missing_gain_data = data[data['GAIN'].isna()].drop(columns=['GAIN', 'PLAYID'])

# Verify the missing_gain_data to ensure it's not empty
print("Shape of missing_gain_data:", missing_gain_data.shape)
if missing_gain_data.empty:
    raise ValueError("missing_gain_data is empty. Check the 'GAIN' column for proper labeling of missing values.")

# Prepare the data
X_known = known_gain_data.drop(columns=['PLAYID', 'GAIN'])
y_known = known_gain_data['GAIN']

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_known, y_known, test_size=0.2, random_state=42)

# Fit the preprocessor on the training data
preprocessor.fit(X_train)
X_train_preprocessed = preprocessor.transform(X_train)
X_val_preprocessed = preprocessor.transform(X_val)

# Save the preprocessor
joblib.dump(preprocessor, 'preprocessor.pkl')

# Define models
models = {
    'RandomForest': RandomForestRegressor(random_state=42),
    'XGBoost': XGBRegressor(random_state=42),
    'CatBoost': CatBoostRegressor(random_state=42, verbose=0),
    'GradientBoosting': GradientBoostingRegressor(random_state=42)
}

# Cross-validation and model selection
best_rmse = float('inf')
best_model_name = None
best_model = None

# Use StratifiedKFold for cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for name, model in models.items():
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])
    # Note: We need to bin the target variable to use StratifiedKFold
    y_train_binned = pd.cut(y_train, bins=5, labels=False)
    scores = cross_val_score(pipeline, X_train, y_train, cv=cv, scoring='neg_root_mean_squared_error')
    mean_rmse = -scores.mean()
    print(f'{name} RMSE: {mean_rmse}')
    if mean_rmse < best_rmse:
        best_rmse = mean_rmse
        best_model_name = name
        best_model = pipeline  # Use pipeline as the best model

print(f'Best model: {best_model_name} with RMSE: {best_rmse}')

# Hyperparameter tuning for the best model
if best_model_name == 'RandomForest':
    param_grid = {
        'regressor__n_estimators': [50, 100, 200],
        'regressor__max_depth': [None, 10, 20],
        'regressor__min_samples_split': [2, 5, 10],
        'regressor__min_samples_leaf': [1, 2, 4]
    }
elif best_model_name == 'XGBoost':
    param_grid = {
        'regressor__n_estimators': [50, 100, 200],
        'regressor__max_depth': [3, 5, 7],
        'regressor__learning_rate': [0.01, 0.1, 0.2],
        'regressor__colsample_bytree': [0.8, 0.9, 1.0]
    }
elif best_model_name == 'CatBoost':
    param_grid = {
        'regressor__iterations': [100, 200, 300],
        'regressor__depth': [4, 6, 10],
        'regressor__learning_rate': [0.01, 0.1, 0.2],
        'regressor__l2_leaf_reg': [1, 3, 5]
    }
elif best_model_name == 'GradientBoosting':
    param_grid = {
        'regressor__n_estimators': [50, 100, 200],
        'regressor__max_depth': [3, 5, 7],
        'regressor__learning_rate': [0.01, 0.1, 0.2],
        'regressor__subsample': [0.8, 0.9, 1.0]
    }

random_search = RandomizedSearchCV(best_model, param_distributions=param_grid, n_iter=50, cv=cv, verbose=2, n_jobs=-1, random_state=42, scoring='neg_root_mean_squared_error')
random_search.fit(X_train, y_train)

# Get the best model
best_model = random_search.best_estimator_

# Validate the best model
y_pred = best_model.predict(X_val)
rmse = root_mean_squared_error(y_val, y_pred)
print(f'Validation RMSE after hyperparameter tuning: {rmse}')

# Save the model for later use
joblib.dump(best_model, 'gain_prediction_model.pkl')

Shape of missing_gain_data: (125, 14)
RandomForest RMSE: 9.533683069246973
XGBoost RMSE: 9.988619782439923
CatBoost RMSE: 9.21400541937754
GradientBoosting RMSE: 9.216299028899016
Best model: CatBoost with RMSE: 9.21400541937754
Fitting 5 folds for each of 50 candidates, totalling 250 fits
Validation RMSE after hyperparameter tuning: 5.677622697916728


['gain_prediction_model.pkl']

In [8]:
# Load the entire pipeline (preprocessor + model)
best_model = joblib.load('gain_prediction_model.pkl')

# Predict the GAIN values for the missing rows using the entire pipeline
predicted_gain = best_model.predict(missing_gain_data)

# Calculate the prediction interval for the sum of GAIN values using bootstrapping
n_iterations = 1000
bootstrapped_sums = []

for _ in range(n_iterations):
    # Resample with replacement
    bootstrapped_sample = resample(predicted_gain)
    bootstrapped_sums.append(bootstrapped_sample.sum())

# Calculate the 5th and 95th percentiles for the 90% prediction interval
lower_bound = np.percentile(bootstrapped_sums, 5)
upper_bound = np.percentile(bootstrapped_sums, 95)

sum_gain = predicted_gain.sum()
print(f'Sum of predicted GAIN: {sum_gain}')
print(f'90% prediction interval for the sum of GAIN: ({lower_bound}, {upper_bound})')

# Store PLAYID values before replacing missing GAIN values
missing_playids = data.loc[data['GAIN'].isna(), 'PLAYID'].values

# Insert the predicted values back into the original dataset
data.loc[data['GAIN'].isna(), 'GAIN'] = predicted_gain

# Create the output DataFrame
output = pd.DataFrame({
    'PLAYID': missing_playids,
    'GAIN': predicted_gain
})

# Display the first few predictions for missing GAIN values
print("First few predictions for missing GAIN values:")
print(output.head())


Sum of predicted GAIN: 673.3729909057872
90% prediction interval for the sum of GAIN: (587.6185962506834, 779.7730884672986)
First few predictions for missing GAIN values:
   PLAYID      GAIN
0     501  5.051178
1     502  0.985289
2     503  8.314523
3     504  1.455107
4     505 -0.980705


In [9]:
# Define individual models
random_forest = RandomForestRegressor(random_state=42)
xgboost = XGBRegressor(random_state=42)
catboost = CatBoostRegressor(random_state=42, verbose=0)
gradient_boosting = GradientBoostingRegressor(random_state=42)

# Hyperparameter tuning for individual models
param_grid = {
    'random_forest': {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    },
    'xgboost': {
        'n_estimators': [50, 100, 200],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.1, 0.2],
        'colsample_bytree': [0.8, 0.9, 1.0]
    },
    'catboost': {
        'iterations': [100, 200, 300],
        'depth': [4, 6, 10],
        'learning_rate': [0.01, 0.1, 0.2],
        'l2_leaf_reg': [1, 3, 5]
    },
    'gradient_boosting': {
        'n_estimators': [50, 100, 200],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.1, 0.2],
        'subsample': [0.8, 0.9, 1.0]
    }
}

models = {
    'random_forest': random_forest,
    'xgboost': xgboost,
    'catboost': catboost,
    'gradient_boosting': gradient_boosting
}

best_estimators = {}

for name, model in models.items():
    search = RandomizedSearchCV(model, param_distributions=param_grid[name], n_iter=50, cv=5, verbose=2, n_jobs=-1, random_state=42, scoring='neg_root_mean_squared_error')
    search.fit(X_train_preprocessed, y_train)
    best_estimators[name] = search.best_estimator_
    print(f'Best {name} model: {search.best_estimator_}')

# Create the stacking ensemble
stacking_ensemble = VotingRegressor(estimators=[
    ('random_forest', best_estimators['random_forest']),
    ('xgboost', best_estimators['xgboost']),
    ('catboost', best_estimators['catboost']),
    ('gradient_boosting', best_estimators['gradient_boosting'])
])

# Fit the ensemble model
stacking_ensemble.fit(X_train_preprocessed, y_train)

# Validate the ensemble model
y_pred = stacking_ensemble.predict(X_val_preprocessed)
rmse = root_mean_squared_error(y_val, y_pred)
print(f'Validation RMSE with Stacking Ensemble: {rmse}')

# Save the ensemble model
joblib.dump(stacking_ensemble, 'stacking_ensemble_model.pkl')

# Load the entire pipeline (preprocessor + ensemble model)
best_model = joblib.load('stacking_ensemble_model.pkl')

# Predict the GAIN values for the missing rows using the entire pipeline
missing_gain_data_preprocessed = preprocessor.transform(missing_gain_data)
predicted_gain = best_model.predict(missing_gain_data_preprocessed)

# Ensure the lengths match before assigning
assert len(predicted_gain) == len(missing_gain_indices), "Length mismatch between predictions and missing data."

# Insert the predicted values back into the original dataset
data.loc[missing_gain_indices, 'GAIN'] = predicted_gain

# Calculate the prediction interval for the sum of GAIN values using bootstrapping
n_iterations = 1000
bootstrapped_sums = []

for _ in range(n_iterations):
    # Resample with replacement
    bootstrapped_sample = resample(predicted_gain)
    bootstrapped_sums.append(bootstrapped_sample.sum())

# Calculate the 5th and 95th percentiles for the 90% prediction interval
lower_bound = np.percentile(bootstrapped_sums, 5)
upper_bound = np.percentile(bootstrapped_sums, 95)

sum_gain = predicted_gain.sum()
print(f'Sum of predicted GAIN: {sum_gain}')
print(f'90% prediction interval for the sum of GAIN: ({lower_bound}, {upper_bound})')

# Store PLAYID values before replacing missing GAIN values
missing_playids = data.loc[missing_gain_indices, 'PLAYID'].values

# Create the output DataFrame
ensemble_output = pd.DataFrame({
    'PLAYID': missing_playids,
    'GAIN': predicted_gain
})

# Display the first few predictions for missing GAIN values
print("First few predictions for missing GAIN values:")
print(ensemble_output.head())


Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best random_forest model: RandomForestRegressor(max_depth=10, min_samples_leaf=4, n_estimators=50,
                      random_state=42)
Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best xgboost model: XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=1.0, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.2, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=50, n_jobs=None,
             num_parallel_tree=None, ra

In [10]:
# Save the predictions to a CSV file
ensemble_output.to_csv('gain_predictions_ensemble_output.csv', index=False)
